In [1]:
import datafaucet as dfc

In [2]:
# start the engine
project = dfc.project.load()

created SparkEngine
Init engine "spark"
Connecting to spark master: local[*]
Engine context spark:2.4.4 successfully started


In [3]:
spark = dfc.context()

In [70]:
df = spark.range(1000)

In [81]:
df = (df
    .cols.create('g').randint(0,5)
    .cols.create('v').randint(0,1000)
    .cols.create('x').randint(0,1000)
    .cols.create('y').fake('first_name')
)

In [82]:
df.data.grid(5)

In [130]:
None or 'ss'

'ss'

In [87]:
pf = df.data.grid()

In [123]:
pf[['g','x', 'y']].groupby('g').agg('min')

,x,y
g,,
0,4,Alexa
1,3,Aaron
2,4,Aaron
3,4,Alan
4,7,Aaron


In [110]:
stack=True

In [128]:
(df
    .cols.get('x', 'y').groupby('g').agg(['min', F.max], stack=False)
).data.grid()


,g,x_min,x_max,y_min,y_max
0,1,3,990,Aimee,William
1,3,4,999,Aaron,William
2,4,7,998,Aaron,Zoe
3,2,4,996,Aaron,William
4,0,4,999,Abigail,Willie


In [97]:
(df
    .cols.get('x', 'y').agg('distinct', stack=stack)
).data.grid()

,colname,distinct
0,y,358
1,x,647


In [98]:
(df
    .cols.get('x').agg(['count','distinct'], stack=stack)
).data.grid()

,colname,count,distinct
0,x,1000,647


In [108]:
(df
    .cols.get('x', 'y').agg(['count','distinct'], stack=stack)
).data.grid()

,x_count,x_distinct,y_count,y_distinct
0,1000,647,1000,342


In [107]:
(df
    .cols.get('x', 'y').agg({'x':'count','y':'distinct'}, stack=stack)
).data.grid()

,x_count,x_distinct,y_count,y_distinct
0,1000,None,None,363


In [7]:
from pyspark.sql import types as T
from pyspark.sql import functions as F

df.cols.get('v').groupby('g').agg(F.countDistinct, F.approx_count_distinct).data.grid()

,g,colname,countDistinct
0,1,v,14
1,3,v,21
2,4,v,20
3,2,v,23
4,0,v,17


In [34]:
# using hll_init (less efficient)

from datafaucet.spark import aggregations as A
from datafaucet.spark import functions 
(df
    .cols.get('v').hll_init()
    .cols.get('v').groupby('g').agg(A.hll_merge(), stack=False)
    .cols.get('v').hll_count()
    .cols.rename('v', 'count') 
).data.grid()

,g,count
0,1,14
1,3,21
2,4,20
3,2,23
4,0,17


In [62]:
# using hll_init_agg (very efficient: sketch in memory)

from datafaucet.spark import aggregations as A
(df
    .cols.get('v').groupby('g').agg(A.hll_init_agg(), stack=False)
    .cols.get('v').hll_count()
    .cols.rename('v', 'count') 
).data.grid()

,g,count
0,1,14
1,3,21
2,4,20
3,2,23
4,0,17


In [42]:
# using hll_init_agg (very efficient: sketch in memory)

from datafaucet.spark.aggregations import hll_init_agg
from datafaucet.spark.functions import hll_count
(df
    .groupby('g').agg(hll_init_agg()(F.col('v')).alias('v'))
    .select('g', hll_count()(F.col('v')).alias('count'))
).data.grid()

,g,count
0,1,14
1,3,21
2,4,20
3,2,23
4,0,17


In [11]:
#pre-aggregate the cube
df_cube = (df
    .cols.get('v').groupby('g').agg({'v_sketch':A.hll_init_agg()})
    .cols.drop('colname')
)
df_cube.data.grid()

,g,v_sketch
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
# re-aqggregate on the cube with hll_merge + hll_count
(df_cube
    .cols.get('v_sketch').hll_count()
    .cols.rename('v_sketch', 'hll_count')
).data.grid()

,g,hll_count
0,1,19
1,3,19
2,4,17
3,2,17
4,0,17


In [13]:
# partial count on the cube: filter + hll_merge + hll_count
(df_cube
     .rows.filter("g in(0,2,4)")
     .cols.get('v_sketch').agg({'v_sketch':A.hll_merge()})
     .cols.get('v_sketch').hll_count()
     .cols.rename('v_sketch','count')
     .cols.drop('colname')
).data.grid()

,count
0,39


In [14]:
# verify the result with countDistinct, and approx_count_distinct
(df
    .rows.filter("g in(0,2,4)")
    .cols.get('v').agg(F.countDistinct, F.approx_count_distinct)
).data.grid()

,colname,<function countDistinct at 0x7f9aff991488>,<function approx_count_distinct at 0x7f9aff9910d0>
0,v,39,41
